<a href="https://colab.research.google.com/github/MWFK/Minervas_V0/blob/main/Avatar_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Winfredy/SadTalker/blob/main/quick_demo.ipynb)

In [ ]:
# @markdown #**Installation of libraries**
# @markdown This cell will take a little while because it has to download several libraries.
%cd "/content"
import requests

print("\nDownloading Packages\n")
# Character Images
!gdown --id "16HzQKA4e3vpLY8Em57WnE8UwIE591aF1" -O "/content/mona_lisa.png" &> /dev/null
!gdown --id "1cgfFgzm4BrqKIkyspGib6u4ty5ReyeM_" -O "/content/einstein.png" &> /dev/null
!gdown --id "10N3e5E0R1aYcLVmE_dmtMCSYVFGQLTeq" -O "/content/lincoln.png" &> /dev/null
!gdown --id "1-BeSNGGjJADs5W-Rn6izAteuVzJcnhW1" -O "/content/nietzsche.png" &> /dev/null
!gdown --id "1zPPUQ7xgbhnpVNl26J1Gl6rXlJ6g0rK7" -O "/content/sokrates.png" &> /dev/null
!gdown --id "1mzzEdXEOohLcpr8L01JzOVbirEMJogni" -O "/content/van_gogh.png" &> /dev/null

# Face Cropping
!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml" -O "/content/haarcascade_frontalface_alt2.xml" &> /dev/null

# Wav2Lip
!git clone "https://github.com/Rudrabha/Wav2Lip.git"
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth" &> /dev/null

try:
  !curl -L -o '/content/Wav2Lip/checkpoints/wav2lip_gan.pth' 'https://drive.google.com/u/0/uc?id=1IKhxXy0mplOpGFWLH9_uUhBoIplao8j0&export=download&confirm=t'
except Exception as e: print(e)
#!gdown --id "1IKhxXy0mplOpGFWLH9_uUhBoIplao8j0" -O "/content/Wav2Lip/checkpoints/wav2lip_gan.pth" &> /dev/null

# First-Order-Model
!git clone "https://github.com/AliaksandrSiarohin/first-order-model"
try:
  !curl -L -o '/content/first-order-model/vox-cpk.pth.tar' 'https://drive.google.com/u/0/uc?id=19d9ZJYAMsNNQZd4AzIWCw4sF1EaNYuJ3&export=download&confirm=t'
except Exception as e: print(e)

# Template Data
#!gdown --id "1Qod7I5hiK1nCPsHBqAdK6hoYZgNzQPHi" -O "driving_video_long.mp4"
!gdown --id "1o2zD5xky8F6wZ21PkeG5KhJOlSdkeEpm" -O "driving_video.mp4" &> /dev/null

# Watermark
url = 'https://raw.githubusercontent.com/mitmedialab/AI-generated-characters/main/gen.png'
r = requests.get(url, allow_redirects=True)
open('gen.png', 'wb').write(r.content)

# Noise
url = 'https://raw.githubusercontent.com/mitmedialab/AI-generated-characters/main/noise2.jpg'
r = requests.get(url, allow_redirects=True)
open('noise_2.png', 'wb').write(r.content)


print("\nInstalling required libraries\n")
!pip install -r Wav2Lip/requirements.txt -y &> /dev/null
!pip uninstall tensorflow tensorflow-gpu -y &> /dev/null
!pip install ffmpeg -y &> /dev/null
!pip install https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip &> /dev/null


# General Functions
print("\nLoading Libraries and functions\n")
import sys
import numpy as np
import ipywidgets as widgets
from io import StringIO
from IPython import get_ipython
from IPython.display import display, Audio, clear_output
from dl_colab_notebooks.audio import record_audio, upload_audio
from scipy.io import wavfile

class IpyExit(SystemExit):
    """
    Exit Exception for IPython.
    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        print("Error: Please only select one input. If you will not use text please leave text field empty.")
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup

from google.colab import files
def getLocalFiles():
  uploaded = files.upload()
  filename = next(iter(uploaded))
  return filename


# First-order-model
import imageio
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def _compute_embedding(audio):
    display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
    global embedding
    embedding = None
    embedding = encoder.embed_utterance(encoder.preprocess_wav(audio, SAMPLE_RATE))

def _record_audio(b):
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  #_compute_embedding(audio)
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  wavfile.write('driving_audio.wav', SAMPLE_RATE, (32767*audio).astype(np.int16))

def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)

def trim_img(img_src):

  import imutils

  # Read the Input Image
  img = cv2.imread(img_src)
  img = imutils.resize(img, width=400)

  # Convert into grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Trim to 400x400
  face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_alt2.xml')
  faces = face_cascade.detectMultiScale(gray, 1.1, 4)
  try:
    for (x, y, w, h) in faces:
      extention = 40
      faces = img[y-extention:y + h+extention, x-extention:x + w + extention]
      cv2.imwrite('/content/img_trimmed.png', faces)
  except:
    print("Error: Face takes too much space on image. Try a different image, or trim it yourself to 400x400.")

  return "/content/img_trimmed.png"


def animate_video(img_filename, vid_filename):
    %cd /content/first-order-model/

    from demo import make_animation
    from demo import load_checkpoints
    from skimage import img_as_ubyte

    source_image = imageio.imread(img_filename)
    driving_video = imageio.mimread(vid_filename, fps=30, memtest=False)

    # Resize image and video to 256x256
    source_image = resize(source_image, (256, 256))[..., :3]
    driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

    # Load Model
    generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='/content/first-order-model/vox-cpk.pth.tar')

    # Make Animation
    predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                                adapt_movement_scale=False)
    #save resulting video
    imageio.mimsave('/content/vidvid.mp4', [img_as_ubyte(frame) for frame in predictions], fps=30)

    %cd /content


def tracability(video_filename):
  import moviepy.editor as mp

  video = mp.VideoFileClip(video_filename)

  machine = (mp.ImageClip('/content/noise_2.png')
    .set_duration(video.duration)
    .set_opacity(.05)
    .resize(height = 552) #
    .margin(right = 0, top = 0, opacity = 1.0)
    .set_pos(("center", "center")))

  human = (mp.ImageClip('/content/gen.png')
   .set_duration(video.duration)
   .resize(height = 50) #
   .margin(right = 0, top = 0, opacity = 1.0)
   .set_pos(("left", "bottom")))

  final = mp.CompositeVideoClip([video, machine, human])
  final.write_videofile("/content/marked.mp4")

print("\nSuccesfully Finished Installing Libraries\n")

In [ ]:
#@markdown #**Choose Inputs**
# @markdown Please select one of the available inputs. Leave the text field empty if you want to animate the character with audio or video.


#Welcome. Today we will learn about the Theory of Relativity. I first came up with this method when...
text = "" #@param {type:"string"}
#@markdown --
audio = True #@param {type:"boolean"}
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav)
record_or_upload = "Upload (.mp3 or .wav)" #@param ["Record", "Upload (.mp3 or .wav)"]
record_seconds =  10#@param {type:"number", min:1, max:10, step:1}
#@markdown --
video = False #@param {type:"boolean"}

if text != "" and audio or text !="" and video or audio and video:
  raise IpyExit


if video:
  print("Please upload the video you wish to drive the animation with:\n")
  video_driver = "/content/"+getLocalFiles()

  #to do: make sure only supported video formats can be uploaded

elif audio:

  SAMPLE_RATE = 22050
  embedding = None

  if record_or_upload == "Record":
    print("Please record the audio you wish to drive the animation with. Remember to enable your microphone in Chrome:\n")
    button = widgets.Button(description="Record Your Voice")
    button.on_click(_record_audio)
    display(button)
    audio_driver = "/content/driving_audio.wav"
  else:
    print("Please upload the audio you wish to drive the animation with:\n")
    audio_driver = "/content/"+getLocalFiles()
  video_driver = "/content/driving_video.mp4"

elif text:
  print("Text is currently unsupported but will be soon.. Please use either audio or video inputs for now.")

Please upload the audio you wish to drive the animation with:



### Installation (around 5 mins)

In [1]:
### make sure that CUDA is available in Edit -> Nootbook settings -> GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


In [ ]:
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1

!sudo apt install python3.8
!sudo apt-get install python3.8-distutils
!python --version

!apt-get update
!apt install software-properties-common

!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

print('Git clone project and install requirements...')
!git clone https://github.com/MWFK/Minervas_V0 &> /dev/null
%cd Minervas_V0
!export PYTHONPATH=/content/Minervas_V0:$PYTHONPATH
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update

!apt install ffmpeg &> /dev/null
!python3.8 -m pip install -r requirements.txt

### Download models (1 mins)

In [ ]:
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

### Choose an Image

In [ ]:
#@markdown
image = True #@param {type:"boolean"}

#@markdown * Either choose a picture from the menu or upload one (.png)
choose_or_upload = "Choose" #@param ["Choose", "Upload (.png)"]

if image:

  if choose_or_upload == "Choose":
    # borrow from makeittalk
    import ipywidgets as widgets
    import glob
    import matplotlib.pyplot as plt

    print("Choose the image name to animate: (saved in folder 'examples/')")
    img_list = glob.glob1('examples/source_image', '*.png')
    img_list.sort()
    img_list = [item.split('.')[0] for item in img_list]
    default_head_name = widgets.Dropdown(options=img_list, value='MK_0')

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            plt.imshow(plt.imread('examples/source_image/{}.png'.format(default_head_name.value)))
            plt.axis('off')
            plt.show()

    default_head_name.observe(on_change)
    display(default_head_name)
    plt.imshow(plt.imread('examples/source_image/{}.png'.format(default_head_name.value)))
    plt.axis('off')
    plt.show()

  else:
    from google.colab import files
    import shutil

    # Upload the picture
    uploaded = files.upload()

    # Specify the destination folder path
    destination_folder = '/content/Minervas_V0/examples/source_image'

    # Move the uploaded file(s) to the destination folder
    for filename in uploaded.keys():
        shutil.move(filename, destination_folder)

### Data Processing

In [ ]:
!pip install openai

In [ ]:
import os
import time
import json
from pathlib import Path
from google.colab import drive
from IPython.display import Audio

import openai
from openai import OpenAI

In [ ]:
drive.mount('/content/drive')

# Load API key from configuration file
config_path = '/content/drive/MyDrive/Colab_Notebooks/API_Keys/openai_api_key.json'  # Update with your path
with open(config_path, 'r') as f:
    config = json.load(f)

api_key = config.get('api_key', None)

if api_key is None:
    raise ValueError("API key not found in the configuration file.")

print("API key loaded successfully.")

In [ ]:
os.environ["OPENAI_API_KEY"] = api_key
print(openai.__version__)
client = OpenAI()

### Convert Audio to text

In [ ]:
audio_file= open("/content/driving_audio.wav", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file,
  response_format="text"
)

transcript

### Text question to text answer

In [ ]:
def getAnswer(instruction, question):

  completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": instruction},
      {"role": "user", "content": question}],
    max_tokens=10,
  )

  return completion

In [ ]:
instruction = "You are an artificial intelligence and data strategy expert, skilled in explaining complex programming and technical concepts with precise flair."
question    = transcript

completion = getAnswer(instruction, question)
answer = completion.choices[0].message.content

print(f"This is your answer: {answer}")

### Text answer to audio answer

In [ ]:
def getAudio(answer):

    response = client.audio.speech.create(
    model="tts-1",
    voice="echo",
    input=answer
    )

    return response

In [ ]:
### Test audio

audio_response = getAudio(answer)

speech_file_path = Path('content').parent / 'audio_answer.mp3'
audio_response.stream_to_file(speech_file_path)

audio = Audio(speech_file_path)
audio

### Inference

In [ ]:
%%time
# selected audio from exmaple/driven_audio
img = 'examples/source_image/{}.png'.format(default_head_name.value)
print(img)
!python3.8 inference.py --driven_audio ./examples/driven_audio/MK_audio_question.wav \
           --source_image {img} \
           --result_dir ./results --still --preprocess full --enhancer gfpgan

### Visualisation

In [14]:
# visualize code from makeittalk
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results

results = sorted(os.listdir('./results/'))

mp4_name = glob.glob('./results/*.mp4')[0]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))


Display animation: ./results/2024_02_07_14.05.50.mp4
